In [119]:
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import boto3, os
from sklearn.metrics import precision_recall_fscore_support

role = get_execution_role()

bucket= 'customer-qqnl'
folder = 'publicis'
#path ='s3://aws-athena-query-results-084313272408-us-east-1/Unsaved/2019/06/23/15099792-408d-4ab2-aa0d-84c251c80e33.csv'


In [5]:
ip_pattern = re.compile('(\d+\.\d+\.\d+\.\d+)')
ip_pattern

re.compile(r'(\d+\.\d+\.\d+\.\d+)', re.UNICODE)

In [3]:
import boto3
import re
import requests
from requests_aws4auth import AWS4Auth

region = 'us-west-1' # e.g. 
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


In [5]:
data_save_location = 's3://{}/{}'.format(bucket, folder)
data_save_location

's3://customer-qqnl/publicis'

In [6]:
#!aws s3 cp balanced.csv $data_save_location/balanced.csv

In [7]:
#path = 's3://aws-athena-query-results-084313272408-us-east-1/Unsaved/2019/06/23/balanced.csv'
df=pd.read_csv('balanced.csv')

In [8]:
df = df.drop(columns=['c0','c1','c2','c3','c4'])

# train test split

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [38]:
x = df.drop(columns='label')

In [39]:
y = df['label']

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x.astype('float32'), y.astype('float32'), test_size =0.1)

In [41]:
train_data = pd.concat([y_train, x_train], axis=1)

In [42]:
train_data.to_csv('train.csv', index=False, header=False)

In [172]:
val_data = pd.concat([y_test, x_test], axis=1)
val_data.to_csv('validation.csv', index=False, header=False)

In [175]:
x_train.shape

(2262918, 303)

In [100]:
test_data = pd.concat([x_test], axis=1)
test_data.to_csv('test.csv', index=False, header=False)

# Upload

In [177]:
output_location = 's3://{}/{}/output'.format(bucket, folder)
data_location = 's3://{}/{}'.format(bucket, folder)
key_train = 'train.csv'
key_test = 'test.csv'
key_validate = 'validation.csv'
#boto3.resource('s3').Bucket(bucket).Object(os.path.join(folder, key_train)).upload_fileobj(buf_train)
#boto3.resource('s3').Bucket(bucket).Object(os.path.join(folder, key_test)).upload_fileobj(buf_test)

In [49]:
!aws s3 cp train.csv $data_location/train.csv

upload: ./train.csv to s3://customer-qqnl/publicis/train.csv       


In [176]:
!aws s3 cp validation.csv $data_location/validation.csv

upload: ./validation.csv to s3://customer-qqnl/publicis/validation.csv


In [101]:
!aws s3 cp test.csv $data_location/test.csv

upload: ./test.csv to s3://customer-qqnl/publicis/test.csv          


# train

In [70]:
import sagemaker

from sagemaker.amazon.amazon_estimator import get_image_uri


container = get_image_uri('us-east-1', 'xgboost')


sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(container, 
                                  role, 
                                  train_instance_count=1,
                                   #train_instance_type='local',
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_location,
                                   #metric_definitions=[]
                                   #content_type='text/csv',
                                   sagemaker_session=sess)
xgb.set_hyperparameters(eval_metric='error',
                      objective='reg:logistic',
                      num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)



In [71]:
s3_train_data = 's3://{}/{}/{}'.format(bucket, folder, key_train)
s3_test_data = 's3://{}/{}/{}'.format(bucket, folder, key_test)

#s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train.csv'.format(bucket, folder), content_type='csv')
#s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/test.csv/'.format(bucket, folder), content_type='csv')

#xgb.fit({'train': s3_input_train,'validation':s3_input_validation}, ContentType='text/csv')

In [81]:
bucket + "/"+ folder + "/output"

'customer-qqnl/publicis/output'

In [82]:

#Ensure that the train and validation data folders generated above are reflected in the "InputDataConfig" parameter below.
common_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "TrainingJobName":'xgboost-06-26',
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": "s3://"+bucket + "/"+ folder + "/output"
    },
    "ResourceConfig": {
        "InstanceCount": 1,   
        "InstanceType": "ml.m4.10xlarge",
        "VolumeSizeInGB": 5
    },
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "silent":"0",
        "objective": "reg:logistic",
        "num_round": "100"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train_data,
                    "S3DataDistributionType": "FullyReplicated" 
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_test_data,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        }
    ]
}

In [83]:
sm = boto3.Session().client('sagemaker')

sm.create_training_job(**common_training_params)

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:084313272408:training-job/xgboost-06-26',
 'ResponseMetadata': {'RequestId': '1ea51097-42cd-40f6-8ed6-e4e892597a3f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1ea51097-42cd-40f6-8ed6-e4e892597a3f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Wed, 26 Jun 2019 17:50:56 GMT'},
  'RetryAttempts': 0}}

In [86]:
status = sm.describe_training_job(TrainingJobName='xgboost-06-26')['TrainingJobStatus']
print(status)

Completed


In [20]:
# fm_predictor = fm.deploy(initial_instance_count=1,
#                          instance_type='ml.m4.xlarge')

---------------------------------------------------------------------------------------------------!

In [94]:
%%time
import boto3
from time import gmtime, strftime

model_name='xgboost-06-26'
info = sm.describe_training_job(TrainingJobName=model_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

s3://customer-qqnl/publicis/output/xgboost-06-26/output/model.tar.gz
arn:aws:sagemaker:us-east-1:084313272408:model/xgboost-06-26
CPU times: user 13.2 ms, sys: 3.88 ms, total: 17.1 ms
Wall time: 322 ms


# Batch transform

In [90]:
batch_output

's3://customer-qqnl/publicis/batch-inference'

In [103]:
model_name='xgboost-06-26'
batch_job_name='batch-xgboost-06-26-01'
batch_output = 's3://{}/{}/batch-inference'.format(bucket, folder) # The location to store the

request = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": s3_test_data 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m4.xlarge",
            "InstanceCount": 1
    }
}

In [104]:
sm.create_transform_job(**request)
import time                            
while(True):
    response = sm.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Transform job is still in status: " + status)    
    time.sleep(30) 



Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job ended with status: Completed


In [107]:
!aws s3 cp --recursive $batch_output ./

download: s3://customer-qqnl/publicis/batch-inference/test.csv.out to ./test.csv.out
download: s3://customer-qqnl/publicis/batch-inference/recordio_test.out to ./recordio_test.out


In [108]:
!head test.csv.out

0.552810668945
0.743256866932
0.402702689171
0.919837415218
0.597523093224
0.671591997147
0.345740288496
0.213444679976
0.422062963247
0.591651439667


In [112]:
pred = pd.read_csv('test.csv.out', header=None)

In [113]:
pred.shape

(251436, 1)

In [114]:
y_test.shape

(251436,)

In [117]:
pred[pred >0.5]=1
pred[pred <=0.5]=0

In [120]:
print(confusion_matrix(y_true=y_test, y_pred=pred))
precision_recall_fscore_support(y_test, pred)

[[87552 37733]
 [47723 78428]]


(array([0.64721493, 0.67516636]),
 array([0.69882268, 0.62169939]),
 array([0.67202947, 0.64733071]),
 array([125285, 126151]))

In [123]:
precision_recall_fscore_support(y_test, pred, average='weighted')

(0.6612387836248483, 0.6601282234843061, 0.6596375606099469, None)

# auto tuning

In [183]:

#Ensure that the train and validation data folders generated above are reflected in the "InputDataConfig" parameter below.
s3_validation_data = 's3://{}/{}/{}'.format(bucket, folder, key_validate)

from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(boto3.Session().region_name, 'xgboost')

training_job_definition = \
{
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
   # "TrainingJobName":'xgboost-06-26',
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": "s3://"+bucket + "/"+ folder + "/output"
    },
    "ResourceConfig": {
        "InstanceCount": 1,   
        "InstanceType": "ml.m4.10xlarge",
        "VolumeSizeInGB": 5
    },
    "StaticHyperParameters": {
        "eval_metric":'error',
        "gamma":"4",
        "objective": "reg:logistic",
        "num_round": "100"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train_data,
                    "S3DataDistributionType": "FullyReplicated" 
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_validation_data,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        }
    ]
}

In [184]:
from time import gmtime, strftime, sleep
tuning_job_name = 'xgboost-tuningjob-' + strftime("%d-%H-%M-%S", gmtime())
print(tuning_job_name)
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 50,
      "MaxParallelTrainingJobs": 5
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:error",
      "Type": "Minimize"
    }
  }

xgboost-tuningjob-27-02-41-32


In [189]:
smclient = boto3.Session().client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                            HyperParameterTuningJobConfig = tuning_job_config,
                                            TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:084313272408:hyper-parameter-tuning-job/xgboost-tuningjob-27-02-41-32',
 'ResponseMetadata': {'RequestId': 'a9a50907-52cd-4be5-954a-083600aeae93',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a9a50907-52cd-4be5-954a-083600aeae93',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '130',
   'date': 'Thu, 27 Jun 2019 02:46:04 GMT'},
  'RetryAttempts': 0}}

In [166]:
smclient.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name)['HyperParameterTuningJobStatus']

'InProgress'

In [188]:
!aws sagemaker stop-hyper-parameter-tuning-job --hyper-parameter-tuning-job-name xgboost-tuningjob-27-02-35-29

### best job is xgboost-tuningjob-27-02-41-32-050-91e7d346


In [190]:
job_best ='xgboost-tuningjob-27-02-41-32-050-91e7d346'

In [192]:
model_name=job_best
batch_job_name='batch-xgboost-06-27-01'
batch_output = 's3://{}/{}/batch-inference_autotuned'.format(bucket, folder) # The location to store the

#model_name='xgboost-06-26'
info = sm.describe_training_job(TrainingJobName=model_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

s3://customer-qqnl/publicis/output/xgboost-tuningjob-27-02-41-32-050-91e7d346/output/model.tar.gz
arn:aws:sagemaker:us-east-1:084313272408:model/xgboost-tuningjob-27-02-41-32-050-91e7d346


In [193]:
request = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": s3_test_data 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m4.xlarge",
            "InstanceCount": 1
    }
}



sm.create_transform_job(**request)
import time                            
while(True):
    response = sm.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Transform job is still in status: " + status)    
    time.sleep(30) 

Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job ended with status: Completed


In [194]:
!aws s3 cp --recursive $batch_output ./autotuned/

download: s3://customer-qqnl/publicis/batch-inference_autotuned/test.csv.out to autotuned/test.csv.out


In [195]:
pred = pd.read_csv('./autotuned/test.csv.out', header=None)
pred[pred >0.5]=1
pred[pred <=0.5]=0

print(confusion_matrix(y_true=y_test, y_pred=pred))
print(precision_recall_fscore_support(y_test, pred))
precision_recall_fscore_support(y_test, pred, average='weighted')

[[88507 36778]
 [46733 79418]]
(array([0.65444395, 0.68348308]), array([0.7064453 , 0.62954713]), array([0.67945111, 0.65540733]), array([125285, 126151]))


(0.6690135244252712, 0.6678637903880112, 0.6673878127600589, None)

# Backup

In [84]:
batch_output = 's3://{}/{}/batch-inference'.format(bucket, folder) # The location to store the

transformer = fm.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

transformer.transform(data=s3_test_data, data_type='S3Prefix', 
                      content_type='text/csv', split_type='line'
                     )

transformer.wait()

Using already existing model: factorization-machines-2019-06-26-01-46-18-402


..............................................!


In [85]:
!aws s3 cp --recursive $transformer.output_path ./

download: s3://customer-qqnl/publicis/batch-inference/recordio_test.out to ./recordio_test.out


In [86]:
!head recordio_test.out

{"predicted_label":0.0,"score":0.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":1.0,"score":1.0}
{"predicted_label":1.0,"score":1.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":0.0,"score":0.0}
{"predicted_label":1.0,"score":1.0}


In [89]:
y_predicted = []
with jsonlines.open('recordio_test.out') as json_file:
    for row in json_file:
        y_predicted.append(row['predicted_label'])
        
print(confusion_matrix(y_true=y_test, y_pred=y_predicted))
precision_recall_fscore_support(y_test, y_predicted)

[[109949  15439]
 [ 86733  39315]]


(array([0.55901913, 0.71802973]),
 array([0.87687019, 0.31190499]),
 array([0.68276462, 0.43489563]),
 array([125388, 126048]))

In [71]:
import jsonlines
y_predicted = []
with jsonlines.open('recordio_test.out') as json_file:
    for row in json_file:
        y_predicted.append(row['predicted_label'])

In [73]:
confusion_matrix(y_true=y_test, y_pred=y_predicted)

array([[75968, 49420],
       [48412, 77636]])

In [74]:
precision_recall_fscore_support(y_test, y_predicted, ac)

(array([0.61077344, 0.61103765]),
 array([0.6058634 , 0.61592409]),
 array([0.60830851, 0.61347114]),
 array([125388, 126048]))

# auto tuning

In [75]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "0.01",
          "MinValue": "0.0001",
          "Name": "factors_init_sigma"
        },
        {
          "MaxValue": "0.02",
          "MinValue": "0.0001",
          "Name": "linear_init_sigma"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "1000",
          "MinValue": "64",
          "Name": "num_factors"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 30,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "test:binary_classification_accuracy",
      "Type": "Maximize"
    }
  }

In [78]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'num_factors': IntegerParameter(64, 1000)}
objective_metric_name = 'test:binary_classification_accuracy'

tuner = HyperparameterTuner(fm,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=50,
                            max_parallel_jobs=3)

tuner.fit({'train': s3_train_data,'test':s3_test_data})

ClientError: An error occurred (ValidationException) when calling the CreateHyperParameterTuningJob operation: The hyperparameter tuning job that you requested has the following untunable hyperparameters: [num_factors]. For the algorithm, 382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:1, you can tune only [factors_init_value, factors_init_scale, bias_init_sigma, factors_init_sigma, linear_wd, linear_init_sigma, bias_init_value, linear_lr, linear_init_scale, bias_wd, bias_lr, factors_wd, bias_init_scale, epochs, factors_lr, linear_init_value, mini_batch_size]. Delete untunable hyperparameters.

In [92]:
y_true = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
y_pred = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
precision_recall_fscore_support(y_true, y_pred)

(array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([2, 2, 2]))